In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
dropout_value = 0.05
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # in = 28x28x1 , out = 26x26x16, RF = 3

        # CONVOLUTION BLOCK 1
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # in = 26x26x16 , out = 24x24x16, RF = 5
 
        # TRANSITION BLOCK 1
        self.pool1 = nn.MaxPool2d(2, 2) # in = 24x24x16 , out = 12x12x16, RF = 6

        # CONVOLUTION BLOCK 2
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # in = 12x12x16 , out = 10x10x16, RF = 10
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # in = 10x10x16 , out = 8x8x16, RF = 14

        # OUTPUT BLOCK
        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # in = 8x8x16 , out = 6x6x16, RF = 18
        self.Gap1 = nn.Sequential(
            nn.AvgPool2d(kernel_size=6)
        ) # in = 6x6x16 , out = 1x1x16, RF = 28	
        self.fc1 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=10, kernel_size=(1, 1), padding=0, bias=False)
        ) # in = 1x1x16 , out = 1x1x10, RF = 28

    def forward(self, x):
        x = self.convblock2(self.convblock1(x))
        x = self.pool1(x)
        x = self.convblock4(self.convblock3(x))
        x = self.Gap1(self.convblock5(x)) # torch.Size([64, 16, 1, 1])
        x = self.fc1(x) # torch.Size([64, 10, 1, 1])
        x = x.view(-1, 10) # torch.Size([64, 10])
        return F.log_softmax(x, dim=-1) #  torch.Size([64, 10])

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Gap
torch.Size([2, 16, 1, 1])
FC
torch.Size([2, 10, 1, 1])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
           Dropout-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           2,304
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
         MaxPool2d-9           [-1, 16, 12, 12]               0
           Conv2d-10           [-1, 16, 10, 10]           2,304
             ReLU-11           [-1, 16, 10, 10]               0
      BatchNorm2d-12           [-1, 16, 10, 10]              32
          Dropout-13           [-1, 16, 10, 

In [ ]:
torch.manual_seed(1)
batch_size = 64

# transforms and DataLoader were written as seperate functions
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, momentum=0.9)

for epoch in range(0, 20):
    print('Epoch:',epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    break

  0%|          | 0/938 [00:00<?, ?it/s]

Epoch: 0


loss=2.1648025512695312 batch_id=7:   0%|          | 3/938 [00:00<00:32, 29.08it/s]

Gap
torch.Size([64, 16, 1, 1])
FC
torch.Size([64, 10, 1, 1])
Gap
torch.Size([64, 16, 1, 1])
FC
torch.Size([64, 10, 1, 1])
Gap
torch.Size([64, 16, 1, 1])
FC
torch.Size([64, 10, 1, 1])
Gap
torch.Size([64, 16, 1, 1])
FC
torch.Size([64, 10, 1, 1])
Gap
torch.Size([64, 16, 1, 1])
FC
torch.Size([64, 10, 1, 1])
Gap
torch.Size([64, 16, 1, 1])
FC
torch.Size([64, 10, 1, 1])
Gap
torch.Size([64, 16, 1, 1])
FC
torch.Size([64, 10, 1, 1])
Gap
torch.Size([64, 16, 1, 1])
FC
torch.Size([64, 10, 1, 1])
Gap
torch.Size([64, 16, 1, 1])
FC
torch.Size([64, 10, 1, 1])


loss=1.948926329612732 batch_id=11:   1%|          | 10/938 [00:00<00:26, 34.46it/s]

...

loss=0.1222924068570137 batch_id=485:  51%|█████▏    | 481/938 [00:08<00:08, 54.06it/s] 

Gap
torch.Size([64, 16, 1, 1])
FC
torch.Size([64, 10, 1, 1])
Gap
torch.Size([64, 16, 1, 1])
FC
torch.Size([64, 10, 1, 1])
Gap
torch.Size([64, 16, 1, 1])
FC
torch.Size([64, 10, 1, 1])
Gap
torch.Size([64, 16, 1, 1])
FC
torch.Size([64, 10, 1, 1])
Gap
torch.Size([64, 16, 1, 1])
FC
torch.Size([64, 10, 1, 1])
Gap
torch.Size([64, 16, 1, 1])
FC
torch.Size([64, 10, 1, 1])
Gap
torch.Size([64, 16, 1, 1])
FC
torch.Size([64, 10, 1, 1])
Gap
torch.Size([64, 16, 1, 1])
FC
torch.Size([64, 10, 1, 1])


loss=0.07871042191982269 batch_id=489:  52%|█████▏    | 487/938 [00:08<00:08, 54.28it/s]

Gap
torch.Size([64, 16, 1, 1])
FC
torch.Size([64, 10, 1, 1])
Gap
torch.Size([64, 16, 1, 1])
FC
torch.Size([64, 10, 1, 1])
Gap
torch.Size([64, 16, 1, 1])
FC
torch.Size([64, 10, 1, 1])
Gap
torch.Size([64, 16, 1, 1])
FC
torch.Size([64, 10, 1, 1])


KeyboardInterrupt: ignored